In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from LoadResults import loadResults

In [3]:
# Load results
results, decisions = loadResults(folder="Final Results")

In [4]:
results['instance'] = results['instance'].astype(int)
results['theta'] = results['theta'].astype(float)
results_sum = results.groupby(['instance','theta', 'date', 'policy'])[['objective', 'electricity cost', 'discomfort index']].sum().reset_index()
results_mean = results_sum.groupby(['instance','theta', 'policy'])[['objective', 'electricity cost', 'discomfort index']].mean().reset_index()
results_mean.head()

,instance,theta,policy,objective,electricity cost,discomfort index
0,0,0.005,Deterministic,1.407174,125.077679,0.785714
1,0,0.005,Lookahead,1.130398,69.722535,0.785714
2,0,0.005,Oracle,0.628394,-30.678251,0.785714
3,0,0.005,Real,2.791272,558.254341,0.000000
4,0,0.010,Deterministic,2.028134,125.027650,0.785714


In [5]:
fig = make_subplots(rows=3, cols=5, subplot_titles=['House: {}'.format(i) for i in range(len(results_mean.instance.unique()))],
                    x_title="Theta", y_title="Weighted objective")

i=0
colors = px.colors.qualitative.Plotly
color_map = {p: colors[i] for i,p in enumerate(results_mean.policy.unique())}
for row in range(1,4):
    for col in range(1,6):
        
        if i > 12:
            continue
        
        data = results_mean[results_mean.instance==i].drop(columns=['instance'])
        
        for p in data.policy.unique():
            data2 = data[data.policy == p]
            
            fig.add_trace(
                go.Scatter(x=data2.theta, y=data2.objective, legendgroup=p, name=p, line_color=color_map[p], showlegend= not i, line_width=2, marker_size=5),
                row=row, col=col
            )

    
            #fig.update_yaxes(type="log")
        i += 1

fig.update_layout(height=800*0.7, width=1500*0.7, title_text="Weighted objective for each policy", )
fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="right",
        x=1
    ))
fig.for_each_xaxis(lambda axis: axis.title.update(font=dict(size=12)))
fig.for_each_yaxis(lambda axis: axis.title.update(font=dict(size=12)))

fig.show()
fig.write_image("../Soportes documento/Graficas ppt/objective_thetas.png", scale=2, height = 700, width=1300)
fig.write_image("../Soportes documento/Graficas documento/objective_thetas.png", scale=2, height = 800*0.7, width=1500*0.7)

In [6]:
fig = make_subplots(rows=3, cols=5, subplot_titles=['House: {}'.format(i) for i in range(len(results_mean.instance.unique()))],
                    x_title="Discomfort index", y_title="Electricity cost [cents/day]")

i=0
colors = px.colors.qualitative.Plotly
color_map = {p: colors[i] for i,p in enumerate(results_mean.policy.unique())}
for row in range(1,4):
    for col in range(1,6):
        
        if i > 12:
            continue
        
        data = results_mean[results_mean.instance==i].drop(columns=['instance'])
        
        for p in data.policy.unique():

            if p == 'Real':
                continue
                #pass
            data2 = data[data.policy == p]
            
            fig.add_trace(
                go.Scatter(x=data2['discomfort index'], y=data2['electricity cost'], legendgroup=p, name=p, line_color=color_map[p], showlegend= not i, line_width=2, marker_size=5),
                row=row, col=col
            )
       
        i += 1

fig.update_layout(height=800*0.7, width=1500*0.7,title_text="Electricity cost and discomfort index for each policy")
fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="right",
        x=1
    ))
fig.for_each_xaxis(lambda axis: axis.title.update(font=dict(size=12)))
fig.for_each_yaxis(lambda axis: axis.title.update(font=dict(size=12)))
fig.show()
fig.write_image("../Soportes documento/Graficas ppt/objectives_thetas.png", scale=2, height = 700, width=1300)
fig.write_image("../Soportes documento/Graficas documento/both_objectives.png", scale=2, height=800*0.7, width=1500*0.7)

In [7]:
df = results_mean.groupby(['instance', 'policy'])['electricity cost'].mean().reset_index()
fig = px.bar(df, y="electricity cost", x="instance", color='policy', barmode='group', text_auto='.2s')
fig.update_layout(title_text="Electricity cost for each house",xaxis=dict(title='House',dtick= 1))
fig.update_yaxes(title="Electricity cost [cents/kWh]")
fig.show()

In [30]:
df.pivot(index=['policy'], columns='instance', values='electricity cost').reset_index().to_excel("tabla.xlsx")

/var/folders/zj/tfzbsc3d4pjfr_d6pw0fdtgr0000gn/T/ipykernel_8433/3281569146.py:1: UserWarning:

Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).



In [8]:
df = results_mean.groupby(['instance', 'policy']).objective.mean().reset_index()
fig = px.bar(df, y="objective", x="instance", color='policy', barmode='group', text_auto='.2s')
fig.update_layout(title_text="Weighted objective for each house",xaxis=dict(title='House',dtick= 1))
fig.show()
#fig.write_image("../Soportes documento/Graficas ppt/objective_houses.png", scale=2, height = 500, width=1000)

Mejora con respecto al método determinístico

In [9]:
df = results_mean.groupby(['instance', 'policy'])['electricity cost'].mean().reset_index()
results_policies=df.pivot(index=['instance'], columns='policy', values='electricity cost').reset_index()
results_policies['gap_deterministic'] = (results_policies['Deterministic']-results_policies['Lookahead'])/abs(results_policies['Deterministic'])
results_policies['gap_oracle'] = (results_policies['Lookahead']-results_policies['Oracle'])/abs(results_policies['Oracle'])
results_policies.head()

policy,instance,Deterministic,Lookahead,Oracle,Real,gap_deterministic,gap_oracle
0,0,90.712225,21.355336,-77.909770,558.254341,0.764581,1.274103
1,1,-218.061856,-294.301652,-380.784935,383.846073,0.349625,0.227118
2,2,-437.251020,-564.972079,-656.297113,881.616275,0.292100,0.139152
3,3,-577.804780,-722.890456,-886.896484,1640.464962,0.251098,0.184921
4,4,-178.792646,-264.530216,-361.247464,670.286994,0.479536,0.267731


In [10]:
gaps = results_policies[['instance', 'gap_deterministic', 'gap_oracle']].melt(id_vars=['instance'])
#gaps['theta'] = gaps['theta'].astype(str)
gaps['value'] = gaps['value']*100
gaps.head()

,instance,policy,value
0,0,gap_deterministic,76.458150
1,1,gap_deterministic,34.962463
2,2,gap_deterministic,29.210008
3,3,gap_deterministic,25.109809
4,4,gap_deterministic,47.953633


In [11]:
fig = px.bar(gaps[gaps.policy=="gap_deterministic"], x="instance", y="value", text_auto='.2s')
fig.update_layout(title_text="Improvement of direct loohahead policy vs deterministic policy", yaxis=dict(title="Improvement (%)"), xaxis=dict(title='House', dtick= 1))
fig.show()
fig.write_image("../Soportes documento/Graficas ppt/gap_deterministic.png", scale=2, height=800*0.7, width=1500*0.7)

In [12]:
instances = pd.read_csv('Data/instances/instances.csv', index_col=0).to_numpy().reshape(14)
instance_dict = {val: idx for idx, val in enumerate(instances)}

for i in instances:
    found_result = True
    try:
        df = pd.read_csv("Results/Lookahead/{i}/errors.csv".format(i=i), index_col=0)
    except:
        found_result = False
    
    if found_result:
        try:
            df_errors = pd.concat([df_errors, df], ignore_index=True)
        except:
            df_errors = df.copy()

df_errors['instance'] = df_errors['instance'].map(instance_dict)

In [13]:
df_errors = df_errors.groupby(['instance']).mean()
gaps = gaps.merge(df_errors, how="left", left_on="instance", right_index=True)

/var/folders/zj/tfzbsc3d4pjfr_d6pw0fdtgr0000gn/T/ipykernel_8433/110866903.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [32]:
data = gaps[gaps['policy']=='gap_deterministic'].copy()
data['value2'] = np.log(data['value'])
#data['instance'] = data['instance'].astype(str)
fig = px.scatter(data, x="g_rmse", y="l_mse", color="value2", text='instance', hover_data='instance')
fig.update_layout(title_text="Improvement over deterministic policy vs forecasting error", yaxis=dict(title="Non-shiftable energy demand MSE"), xaxis=dict(title="PV energy MSE"))
fig.update_traces(textposition='top center')
#fig.write_image("../Soportes documento/Graficas ppt/improvement_vs_forecast.png", scale=2, height = 500, width=1000)

height = 500
width = 600
fig.update_layout(height=height, width=width, title_text="Weighted objective for each policy", )
#fig.write_image("../Soportes documento/Graficas documento/improvement_vs_forecast.png", scale=2, height = height, width=width)
fig.show()

In [33]:
fig = px.scatter(data[data['instance']!=9], x="g_rmse", y="value", text='instance', hover_data='instance')
fig.update_traces(textposition='top center')
height = 500*0.8
width = 600*0.8
fig.update_layout(height=height, width=width,
                  yaxis=dict(title="Improvement over deterministic policy (%)"), xaxis=dict(title="PV energy MSE"))
fig.write_image("../Soportes documento/Graficas documento/improvement_vs_pvmse.png", scale=2, height = height, width=width)
fig.show()

In [22]:
fig = px.scatter(data[data['instance']!=9], x="l_mse", y="value", text='instance', hover_data='instance')
fig.update_layout(title_text="Improvement over deterministic policy vs forecasting error", yaxis=dict(title="Improvement over deterministic policy (%)"), xaxis=dict(title="Non-shiftable energy MSE"))
fig.update_traces(textposition='top center')
fig.show()

In [22]:
fig = px.bar(gaps_mean[gaps_mean.policy=="gap_oracle"], x="instance", y="value", text_auto='.2s')
fig.update_layout(title_text="Gap between loohahead and oracle", yaxis=dict(title="Gap (%)"), xaxis=dict(dtick= 1))
fig.show()
fig.write_image("../Soportes documento/Graficas ppt/gap_oracle.png", scale=2, height = 500, width=1000)

# Duck plot

In [24]:
# Cargar datos de consumo reales
instances = pd.read_csv('Data/instances/instances.csv', index_col=0).to_numpy().reshape(14)
df_real = pd.DataFrame(columns=["local_15min", "grid", "b", "s", "solar", "instance"])

for i in instances:
    df = pd.read_csv("Data/instances/{}/real_{}.csv".format(i,i), parse_dates=[0], sep=",", decimal=".")
    df["instance"] = i
    df_real = pd.concat([df_real, df], ignore_index=True)

df_real['time'] = df_real['local_15min'].dt.strftime('%H:%M')
df_real = df_real[(df_real['local_15min']>="2019-08-01")&(df_real['local_15min']<="2019-08-31")]

In [27]:
#df_real = df_real.groupby(["local_15min", "time"])["grid"].sum().reset_index()
df_real = df_real.groupby(["time"])[["grid", "b", "solar"]].mean().reset_index()

In [29]:
# Cargar datos de consumo
df_dr = decisions[["time", "policy", "theta", "b_t", "s_t", "g_t"]].copy()
df_dr['date'] = pd.to_datetime(df_dr['time'])
df_dr = df_dr[df_dr["theta"]=='0.005']
df_dr['time'] = df_dr['date'].dt.strftime('%H:%M')
df_dr['grid'] = df_dr['b_t'] - df_dr['s_t']
#df_dr = df_dr.groupby(["policy", "date", "time"])["grid"].sum().reset_index()


In [30]:
df_dr = df_dr.groupby(["policy", "time"])[["grid", "b_t", "g_t"]].mean().reset_index()

In [57]:
# Gráfica
fig = px.line(df_dr, x='time', y='grid', color='policy')
fig.add_trace(go.Scatter(x=df_real['time'], y=df_real['grid'], legendgroup="Real", name="Real"))
height = 500*0.8
width = 800*0.8
fig.update_layout(height=height, width=width,
                  yaxis=dict(title="Energy consumption (kWh)"), xaxis=dict(title="Hour"))
fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="right",
        x=1
    ))
fig.write_image("../Soportes documento/Graficas documento/duck_plot.png", scale=2, height = height, width=width)
fig.show()

In [32]:
PAR_real = np.max(df_real['grid'])/np.mean(df_real['grid'])
PAR_look = np.max(df_dr[df_dr['policy']=='Lookahead']['grid'])/np.mean(df_dr[df_dr['policy']=='Lookahead']['grid'])
print("PAR real: "+str(PAR_real))
print("PAR Lookahead: "+str(PAR_look))

PAR real: -4.182759462838979
PAR Lookahead: -10.58873661616763


In [51]:
df_real.describe()

,grid,b,solar
count,96.000000,96.000000,96.000000
mean,-0.075677,0.135546,0.325893
std,0.355722,0.088184,0.393291
min,-0.724588,0.015642,0.000112
25%,-0.404415,0.042506,0.000128
50%,0.137528,0.144416,0.065233
75%,0.187888,0.189000,0.712453
max,0.316537,0.316537,1.029530


In [37]:
sum_real = df_real.sum()
dep_real = sum_real['b']/(sum_real['b']+sum_real['solar'])
dep_real

0.29374563339486487

In [53]:
df_dr[df_dr['policy']=='Lookahead'].describe()

,grid,b_t,g_t
count,96.000000,96.000000,96.000000
mean,-0.094708,0.260111,0.315509
std,0.545136,0.242598,0.380047
min,-1.097579,0.004693,0.000122
25%,-0.588320,0.070329,0.000138
50%,-0.002001,0.195506,0.064204
75%,0.290764,0.344903,0.686324
max,1.002839,1.002839,0.997264


In [41]:
sum_look = df_dr[df_dr['policy']=='Lookahead'].sum()
dep_look = sum_look['b_t']/(sum_look['b_t']+sum_look['g_t'])
dep_look

0.4518802598182404

In [43]:
sum_real

time     00:0000:1500:3000:4501:0001:1501:3001:4502:000...
grid                                             -7.264951
b                                                13.012383
solar                                            31.285751
dtype: object

In [44]:
sum_look

policy    LookaheadLookaheadLookaheadLookaheadLookaheadL...
time      00:0000:1500:3000:4501:0001:1501:3001:4502:000...
grid                                              -9.091973
b_t                                               24.970699
g_t                                               30.288849
dtype: object